##### Modal
# Lab: Ditch Dirty Data

Welcome to your SQL Data Cleaning lab! We are going to work with a dirty dataset from **[monster.com](http://monster.com)** that lists job postings from February 2016 to January 2017. We need a clean dataset so that we can look at manager job trends across the USA for a data visualization project. 

For this lab, we’re just preparing the dataset and not making any visualizations, but it’s important to know why you’re cleaning and how you’ll use the data before you start cleaning it up.


## Introducing SQL

SQL stands for **Structured Query Language**. It's a programming language that lets us store and retrieve important business data stored in _relational databases_. In the most basic terms, you can think of a relational database as a set of spreadsheets linked together. In SQL, we call these spreadsheets **tables**.

### Table Structure

If you've worked with spreadsheets before, SQL tables will look quite familiar. Tables are comprised rows and columns. Each row is called a **record**, and each column represents an **attribute** of that record.

In our example dataset, each row represents a newly posted job listing on monster.com. The columns simply tell us more about that job. Each job might have attributes like `job_title`, `location`, or `salary`:

<img src="./images/sql-tables.png" alt="Drawing" style="display:inline-block; text-align:left; width:100%; max-width: 600px;"/>


### Connecting to our Database

The first step in working with SQL is connecting to our database. To get warmed up, try running the following command to create a new connection to our `jobs` database.

You can use **`SHIFT + RETURN`** to run any SQL code block in this lab.


In [ ]:
## Load SQL
%load_ext sql

## Connect to our SQL database called "jobs"
%sql sqlite:///jobs.db

----

## Querying Our Database

The first step in cleaning our dataset is being able to find the data we care about. SQL lets us write **queries** to do this.

SQL queries generally start with a `SELECT` statement. `SELECT` helps us find the specific rows, or records, that match our query criteria.

Let’s start with a simple query to get a taste of what our dataset looks like:

In [ ]:
%%sql

SELECT * FROM jobs LIMIT 2;

So what's happening here? `SELECT` statements generally follow the format:

>
>```sql
>SELECT [columns] FROM [table_name] ... [QUERY FILTERS]
>```
>

Here, we used the `*` operator as a shortcut for "all columns", and `LIMIT 2` as a query filter to only return the first 2 matching rows in our database.

#### TRY IT

While using the **`*`** operator is a common starting point for queries to help understand what information is contained in the table, it often returns too much data. 

Edit this query to return only the `country_code`, `job_title`, and `location`columns for each job in the database:


In [ ]:
%%sql

SELECT country_code, job_title FROM jobs LIMIT 10;

----

## Filtering Our Data

Great! Now we have the ability to write basic queries, and select specific information to be returned for individual job listings.

But monster.com has thousands of job listings. If we want to answer specific questions about job trends in the US, we need to be able to write more powerful queries using filters.


#### WHERE

Instead of limiting ourselves to the first 2 results with the `LIMIT` filter, let's explore ways to filter our data. We can use the `WHERE` statement to help us give our query specific matching criteria:

A common pattern for `WHERE` filtering is:

```sql
WHERE [column_name] = [value]
```

Let's try it here by looking for the first 10 jobs that haven't expired:

In [ ]:
%%sql

SELECT * FROM jobs WHERE has_expired = 'No' LIMIT 10;

We can use the `AND` operator to make more specific conditions for the `WHERE` statement.

Let's update our query to look for active jobs in the U.S.:

In [ ]:
%%sql

SELECT * FROM jobs WHERE has_expired = 'No' AND country_code = 'US' LIMIT 10;

#### QUERIES ON MULTIPLE LINES

Simple queries are ok on one line. But the more complex your queries get, the tougher it becomes to read and understand what's goin on. It’s easier to space them out over multiple lines for readability. Let’s make this query a little easier to read by re-writing it over multiple lines.

Run this SQL command and compare the output from our last query. They should match!

In [ ]:
%%sql

SELECT * FROM jobs
WHERE has_expired = 'No'
AND country_code = 'US' 
LIMIT 10;

#### Finding Manager Job Listings

Let's dig in to find management job listings. As in the **Try It** above, we'll focus our search on a smaller number of columns: `job_type`, `job_title`, and `location`  

In [ ]:
%%sql

SELECT job_type, job_title, location
FROM jobs
WHERE has_expired = 'No'
AND country_code = 'US' 
LIMIT 10;

Let’s make a first attempt to find _just_ the manager jobs in the dataset to see how manager roles differ by state. 

By adding another `AND` clause, we can search for job titles that are _exactly_ *“Manager”*:

In [ ]:
%%sql

SELECT job_type, job_title, location
FROM jobs
WHERE has_expired = 'No'
AND country_code = 'US'
AND job_title = 'Manager';

That found us a few jobs - but what about all the variations of management job titles that don't _exactly_ match the title "Manager"?

To help find those variations, we can use the `LIKE` operator with a new wildcard operator, `%` to find any job that has the word "Manager" _somewhere_ in the title (e.g. _"Assistant Manager" or "Regional Manager"):

`LIKE` is often used in the following format:

>```sql
>[column name] LIKE `%[search phrase]%`
>```


In [ ]:
%%sql

SELECT job_type, job_title, location
FROM jobs
WHERE has_expired = 'No'
AND country_code = 'US'
AND job_title LIKE '%Manager%';

Much better. But it's easy to think of cases where we still might miss Manager positions. For example, what about all the job titles where the phrase "management" is used? (e.g. _"Operations Management"_)

We could solve this in two ways. One method would be to change our `LIKE` filter to find anything with "Manage" in the job title. Searching for `%Manage` would find both _"**Manage**r"_ and _"Operations **Manage**ment"_ because "Manage" is in both.

But what if there were specific titles we wanted to search for that weren't so similar? 

#### OR

We can use the `OR` operator to return any jobs that match **at least one** of many filtering criteria.

In the previous example, we could look for any jobs that matched `%Manager%` **`OR`** `%Management%`. We often put `OR` statements inside parentheses so that it applies to a specific part of our query, and doesn't break our other `AND` filtering logic:

>```sql
> AND ([condition 1] OR [condition 2])
>```

Let's try it here:

In [ ]:
%%sql

SELECT job_type, job_title, location
FROM jobs
WHERE has_expired = 'No'
AND country_code = 'US'
AND (job_title LIKE '%Manager%' OR job_title LIKE '%Management%');

#### TRY IT

Let's assume that another common way to designate a management job title is to use the phrase **"Lead"**. 

Edit the following query to make sure that you also find all the jobs that include **"Lead"** in the title:

In [ ]:
%%sql

SELECT job_type, job_title, location
FROM jobs
WHERE has_expired = 'No'
AND country_code = 'US'
AND (job_title LIKE '%Manager%' OR job_title LIKE '%Management%');

---

## Handling Inconsistent Data Formatting

A common problem in data clean up is dealing with all the ways that data can be incorrectly entered into a data set. 

In our jobs example, what happens if one management role is entered as **Manager**, but another is **Regional manager**? With our current queries, we'll miss the second job listing because _"manager"_ starts with a lowercase `m`.

#### UPPER

Checking for all the capitalization combinations can be tedious. To get around this, we can use the “upper” function to capitalize all often characters in a column _before_ we do any filter comparisons. This helps us normalize the data input, and remove inconsistencies in capitalization:

>```sql
>UPPER([column name]) LIKE ['%search phrase%']
>```

Let's try using `UPPER` on the `job_title` column and updating our `LIKE` comparison to look for uppercase matches:

In [ ]:
%%sql

SELECT job_type, job_title, location
FROM jobs
WHERE has_expired = 'No'
AND country_code = 'US'
AND (UPPER(job_title) LIKE '%MANAGE%' OR UPPER(job_title) LIKE '%LEAD');


> **TIP**: Watch out for parentheses! `UPPER` uses `()` around a column name, and we have a second set of `()` around our `OR` condition. For SQL to understand our query, every open paren `(` must have a matching closed paren `)`


#### TRY IT

We can use `UPPER` on any column that contains text. Make sure we're finding all the management jobs by:

1. Editing our query to also return the `job_description` column
2. Adding a filter looking for `%MANAGE` in the uppercase `job_description`

In [ ]:
%%sql

SELECT job_type, job_title, location
FROM jobs
WHERE has_expired = 'No'
AND country_code = 'US'
AND (UPPER(job_title) LIKE '%MANAGE%' OR UPPER(job_title) LIKE '%LEAD');

--- 

## Saving Our Results

Are you starting to feel confident you found all your manager jobs? Great!

Now that we've done the work to find all the management jobs, we can save those results so that someone else doesn't have to start from scratch. To do this, we'll **add a column** called `management` which will be `y` if it matches our manager query logic, and `n` otherwise.

To add a column, we use the `ALTER TABLE` SQL command:

>```sql
>ALTER TABLE [table name] ADD [new column name] [new column type]
>```


In [ ]:
%%sql

ALTER TABLE  jobs ADD management text;

Now you have a single, trustworthy management column for each job entry in the dataset! Let’s populate the column with an **`UPDATE`** statement.

We set the `management` column to `y` for any job that matches our filtering criteria:

In [ ]:
%%sql

UPDATE TABLE  jobs SET management = 'y'
WHERE (UPPER(job_description) LIKE '%MANAG%'
OR UPPER(job_title) LIKE '%LEAD%'
OR UPPER(job_description) LIKE '%MANAG%');

Now we have to classify the non-management jobs in your new column with the same update statement but reversing the logic with a **`NOT`** keyword.


In [ ]:
%%sql

UPDATE TABLE  jobs SET management = 'n'
WHERE NOT (UPPER(job_description) LIKE '%MANAG%'
OR UPPER(job_title) LIKE '%LEAD%'
OR UPPER(job_description) LIKE '%MANAG%');

Try out your new `management` column in a query.

In [ ]:
%%sql

SELECT job_type, job_title, location
FROM jobs 
WHERE has_expired = ‘n’ 
AND country_code = ‘US’
AND management = ‘y’;

## Dig Deeper

Let’s try these techniques again. Instead of focusing on manager as a classification, choose another job type - Sales.  Repeat the steps above and classify more data!

You can edit and run the SQL query below as many times as needed.

In [ ]:
%%sql

SELECT *
FROM jobs
WHERE job_title LIKE '%%'

## Takeaway

Working with clean data is a critical foundation of visualizing your data and conveying accurate information. While this lab developed some SQL skills specifically, many of the approaches can be applied to other data analysis tools. Taking the time to think through all the ways your dataset could be hiding information (data consistency, looking across multiple fields, input errors) can help make sure that you have confidence in the data-driven story you're telling. 